In [1]:
from typing import Optional

import geopandas as gpd
import shapely
import shapely.affinity

from shapely_test import shapes

print(f"{shapely.__version__ = }")

shapely.__version__ = '2.0a1'


In [2]:
def assert_geodataframe_equal(left, right):
    left = left.unary_union
    right = right.unary_union
    iou = left.intersection(right).area / left.union(right).area
    assert iou >= 0.999, f"Failed: {iou = :.8F} >= 0.999"

In [3]:
# turn off snap_to_integers for performance testing
generator = shapes.RandomSpotsGenerator(snap_to_integers=False)
expected = generator(seed=0)

# Translate

In [4]:
def translate_affinity(gdf: gpd.GeoDataFrame, xoff: float, yoff: float, inplace=False) -> Optional[gpd.GeoDataFrame]:
    new_geoms = gdf.geometry.apply(lambda g: shapely.affinity.translate(g, xoff, yoff))
    return gdf.set_geometry(new_geoms, inplace=inplace)


def translate_vectorized(gdf: gpd.GeoDataFrame, xoff: float, yoff: float, inplace=False) -> Optional[gpd.GeoDataFrame]:
    new_geoms = shapely.transform(gdf.geometry, lambda x: x + [xoff, yoff])
    return gdf.set_geometry(new_geoms, inplace=inplace)

---

In [5]:
gdf1 = translate_affinity(expected, 1, 1)
gdf = translate_affinity(gdf1, -1, -1)
assert_geodataframe_equal(expected, gdf)

In [6]:
%%timeit
_ = translate_affinity(expected, 1, 1)

28.6 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


---

In [7]:
gdf1 = translate_vectorized(expected, 1, 1)
gdf = translate_vectorized(gdf1, -1, -1)
assert_geodataframe_equal(expected, gdf)

In [8]:
%%timeit
_ = translate_vectorized(expected, 1, 1)

3.88 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Scale

In [9]:
def scale_affinity(gdf: gpd.GeoDataFrame, xfact: float, yfact: float, inplace=False) -> Optional[gpd.GeoDataFrame]:
    new_geoms = gdf.geometry.apply(lambda g: shapely.affinity.scale(g, xfact, yfact, origin=(0, 0)))
    return gdf.set_geometry(new_geoms, inplace=inplace)


def scale_vectorized(gdf: gpd.GeoDataFrame, xfact: float, yfact: float, inplace=False) -> Optional[gpd.GeoDataFrame]:
    new_geoms = shapely.transform(gdf.geometry, lambda x: x * [xfact, yfact])
    return gdf.set_geometry(new_geoms, inplace=inplace)

---

In [10]:
gdf1 = scale_affinity(expected, 2, 2)
gdf = scale_affinity(gdf1, 0.5, 0.5)
assert_geodataframe_equal(expected, gdf)

In [11]:
%%timeit
_ = scale_affinity(expected, 2, 2)

29.4 ms ± 426 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


---

In [12]:
gdf1 = scale_vectorized(expected, 2, 2)
gdf = scale_vectorized(gdf1, 0.5, 0.5)
assert_geodataframe_equal(expected, gdf)

In [13]:
%%timeit
_ = scale_vectorized(expected, 2, 2)

3.84 ms ± 76.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Coord Transform

Test converting between pixel and Lon/Lat coordinate systems

In [14]:
from typing import Callable, Tuple
import numpy as np


class Transformer:
    """Converts WSG-84 to/from pixels. See See https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames"""

    def __init__(self, zoom: int, px_per_tile: int):
        self._zoom = zoom
        self._px_per_tile = px_per_tile
        self._scale = 2 ** zoom

    def lon_lat_to_px__xy(self, lon: np.ndarray, lat: np.ndarray) -> Tuple:
        lat_rad = np.deg2rad(lat)
        xtile = ((lon + 180.0) / 360.0) * self._scale
        ytile = (1.0 - np.arcsinh(np.tan(lat_rad)) / np.pi) / 2.0 * self._scale
        return xtile * self._px_per_tile, ytile * self._px_per_tile

    def px_to_lon_lat__xy(self, x: np.ndarray, y: np.ndarray) -> Tuple:
        xtile, ytile = x / self._px_per_tile, y / self._px_per_tile
        lon = xtile / self._scale * 360.0 - 180.0
        lat_rad = np.arctan(np.sinh(np.pi * (1 - 2 * ytile / self._scale)))
        lat = np.rad2deg(lat_rad)
        return lon, lat

    def lon_lat_to_px__pts(self, pts: np.ndarray) -> np.ndarray:
        lon, lat = pts.T
        return np.asarray(self.lon_lat_to_px__xy(lon, lat)).T

    def px_to_lon_lat__pts(self, pts: np.ndarray) -> np.ndarray:
        x, y = pts.T
        return np.asarray(self.px_to_lon_lat__xy(x, y)).T


transformer = Transformer(zoom=16, px_per_tile=generator.px_per_tile)

In [15]:
def apply(gdf: gpd.GeoDataFrame, func: Callable, inplace=False) -> Optional[gpd.GeoDataFrame]:
    new_geoms = gdf.geometry.apply(lambda g: shapely.ops.transform(func, g))
    return gdf.set_geometry(new_geoms, inplace=inplace)


def vectorized(gdf: gpd.GeoDataFrame, func: Callable, inplace=False) -> Optional[gpd.GeoDataFrame]:
    new_geoms = shapely.transform(gdf.geometry, func)
    return gdf.set_geometry(new_geoms, inplace=inplace)

---

In [16]:
gdf1 = apply(expected, transformer.px_to_lon_lat__xy)
gdf = apply(gdf1, transformer.lon_lat_to_px__xy)
assert_geodataframe_equal(expected, gdf)

In [17]:
%%timeit
_ = apply(expected, transformer.px_to_lon_lat__xy)

233 ms ± 5.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

In [18]:
gdf1 = vectorized(expected, transformer.px_to_lon_lat__pts)
gdf = vectorized(gdf1, transformer.lon_lat_to_px__pts)
assert_geodataframe_equal(expected, gdf)

In [19]:
%%timeit
_ = vectorized(expected, transformer.px_to_lon_lat__pts)

4.59 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
